<a href="https://colab.research.google.com/github/AlexChek51/Wiki_QA/blob/main/Wiki_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install llama_index pyvis Ipython langchain pypdf langchain_community
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-langchain
!pip install langchain-huggingface
!pip install sentencepiece accelerate
!pip install -U bitsandbytes
!pip install peft
!pip install llama-index-readers-wikipedia wikipedia
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vl8ot5bv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vl8ot5bv
  Resolved https://github.com/huggingface/transformers to commit 0fdea8607d7e01eb0e38a1ebeb7feee30a22f0cf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.43.0.dev0-py3-none-any.whl size=9394962 sha256=c0de34a8b6b0f2d3cb561b95a18ed0822377cb80328ead7a8def9efb6561650c
  Stored in directory: /tmp/pip-ephem-wheel-cache-gtvwa4yz/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM


#from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from pyvis.network import Network

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig
from peft import PeftModel, PeftConfig
from huggingface_hub import login

# Авторизация на Hugging Face
HF_TOKEN = ""
login(HF_TOKEN, add_to_git_credential=True)


# Задаем имя модели
MODEL_NAME = "Den4ikAI/FRED-T5-LARGE_text_qa"

# Загружаем базовую модель
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
model.eval()

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [ ]:
import gradio as gr
from llama_index.readers.wikipedia import WikipediaReader
from langchain_huggingface import HuggingFaceEmbeddings
import time

# Инициализация модели и других компонентов
def initialize_model():
    llm = HuggingFaceLLM(
        model=model,
        model_name=MODEL_NAME,
        tokenizer=tokenizer,
        device_map="auto"
    )

    embed_model = LangchainEmbedding(
        HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    graph_store = SimpleGraphStore()
    storage_context = StorageContext.from_defaults(graph_store=graph_store)

    return storage_context

# Функция для загрузки данных из Википедии
def load_wikipedia_data(query, lang_prefix, progress):
    reader = WikipediaReader()
    progress(0.1, desc="Инициализация WikipediaReader")
    try:
        docs = reader.load_data(
            pages=[query],
            lang_prefix=lang_prefix
        )
        progress(0.4, desc="Данные из Википедии загружены")
        return docs
    except Exception as e:
        raise RuntimeError(f"Не удалось загрузить данные из Википедии: {str(e)}")

# Функция для создания графа из загруженных данных
def create_knowledge_graph(docs, storage_context, progress):
    progress(0.5, desc="Создание графа знаний")
    try:
        indexKG = KnowledgeGraphIndex.from_documents(
            documents=docs,
            max_triplets_per_chunk=3,
            show_progress=True,
            include_embeddings=True,
            storage_context=storage_context
        )
        progress(0.8, desc="Граф знаний создан")
        return indexKG
    except Exception as e:
        raise RuntimeError(f"Не удалось создать граф знаний: {str(e)}")

# Функция для подготовки данных
def prepare(query, lang_prefix, progress):
    progress(0.05, desc="Инициализация модели")
    storage_context = initialize_model()
    progress(0.1, desc="Модель инициализирована")
    docs = load_wikipedia_data(query, lang_prefix, progress)
    progress(0.45, desc="Данные из Википедии загружены")
    indexKG = create_knowledge_graph(docs, storage_context, progress)
    progress(0.9, desc="Граф знаний создан")

    # Извлечение текста из документов
    text = "\n".join([doc.text for doc in docs])

    return text, indexKG


# Функция генерации ответа
def generate(prompt):
    data = tokenizer(f"{prompt}", return_tensors="pt").to(model.device)
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    out = tokenizer.decode(output_ids.tolist())
    return out

# Функция для обработки запросов
def process_query(user_query):
    try:
        if graph_state["graph"] is None or graph_state["text"] is None:
            return "Граф данных или текст статьи не созданы. Пожалуйста, сначала выполните парсинг данных."

        text = graph_state["text"]

        # Разбиение текста на фрагменты
        chunk_size = 512  # Размер фрагмента
        chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

        responses = []
        for chunk in chunks:
            # Формирование запроса
            prompt_template = f"""Текст статьи: {chunk}\nВопрос: {user_query}\nОтвет: <extra_id_0>"""

            # Генерация ответа
            response = generate(prompt_template)
            responses.append(response)

        # Объединение ответов в один текст
        combined_text = " ".join(responses)

        # Генерация итогового ответа на основе всех ответов
        final_prompt = f"Текст: {combined_text}\nВопрос: {user_query}\nОтвет: <extra_id_0>"
        final_response = generate(final_prompt)

        return final_response
    except Exception as e:
        raise RuntimeError(f"Не удалось обработать запрос: {str(e)}")


# Gradio интерфейс
def handle_user_query(query, lang_prefix, user_query, progress=gr.Progress(track_tqdm=True)):
    try:
        start_time = time.time()

        # Подготовка данных
        indexKG = prepare(query, lang_prefix, progress)

        # Обработка пользовательского запроса
        progress(0.95, desc="Обработка запроса пользователя")
        response = process_query(indexKG, user_query)

        end_time = time.time()
        elapsed_time = end_time - start_time

        progress(1.0, desc="Завершено")

        return response, f"Время выполнения: {elapsed_time:.2f} секунд"
    except Exception as e:
        return None, f"Произошла ошибка: {str(e)}"

# Создаем состояние для хранения графа
graph_state = {
    "graph": None
}

# Функция для парсинга данных
def parse_data(query, lang_prefix, progress=gr.Progress(track_tqdm=True)):
    try:
        text, indexKG = prepare(query, lang_prefix, progress)
        graph_state["graph"] = indexKG
        graph_state["text"] = text

        return text, "Данные успешно распарсены.\n"
    except Exception as e:
        return "", f"Произошла ошибка при парсинге данных: {str(e)}"


# Функция для отправки запроса
def send_query(user_query):
    try:
        if graph_state["graph"] is None or graph_state["text"] is None:
            return "Граф данных или текст статьи не созданы. Пожалуйста, сначала выполните парсинг данных."

        response = process_query(user_query)
        return response
    except Exception as e:
        return f"Произошла ошибка при обработке запроса: {str(e)}"


# Интерфейс Gradio
with gr.Blocks() as interface:
    with gr.Row():
        query_input = gr.Textbox(lines=1, placeholder="Введите название статьи из Википедии", label="Запрос")
        lang_prefix_input = gr.Textbox(lines=1, placeholder="Введите префикс языка (например, 'ru' для русского)", value="ru", label="Префикс языка")
        parse_button = gr.Button("Парсинг данных")

    with gr.Row():
        user_query_input = gr.Textbox(lines=1, placeholder="Введите ваш запрос", label="Запрос пользователя")
        send_button = gr.Button("Отправить запрос")

    with gr.Row():
        output_textbox = gr.Textbox(label="Ответ")
        status_textbox = gr.Textbox(label="Статус")
        article_textbox = gr.Textbox(label="Текст статьи", lines=10, interactive=False)

    parse_button.click(fn=parse_data, inputs=[query_input, lang_prefix_input], outputs=[article_textbox, status_textbox])
    send_button.click(fn=send_query, inputs=user_query_input, outputs=output_textbox)

# Запуск интерфейса
interface.launch()